## 0. Importy i pobranie danych

In [146]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, precision_score

In [147]:
glass = pd.read_csv('glass.csv')
pd.set_option('display.width', 2137)
glass = glass.drop('Id', axis=1)
print(glass)

          RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  Type
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0     1
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0     1
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0     1
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0     1
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0     1
..       ...    ...   ...   ...    ...   ...   ...   ...  ...   ...
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0     7
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0     7
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0     7
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0     7
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0     7

[214 rows x 10 columns]


## 1. Eksploaracja danych
przedstaw podstawowe dane statystyczne i uwagi dotyczące cech i etykiet
zbioru danych. (10 punktów)

In [148]:
print(glass.describe())

               RI          Na          Mg          Al          Si           K          Ca          Ba          Fe        Type
count  214.000000  214.000000  214.000000  214.000000  214.000000  214.000000  214.000000  214.000000  214.000000  214.000000
mean     1.518365   13.407850    2.684533    1.444907   72.650935    0.497056    8.956963    0.175047    0.057009    2.780374
std      0.003037    0.816604    1.442408    0.499270    0.774546    0.652192    1.423153    0.497219    0.097439    2.103739
min      1.511150   10.730000    0.000000    0.290000   69.810000    0.000000    5.430000    0.000000    0.000000    1.000000
25%      1.516522   12.907500    2.115000    1.190000   72.280000    0.122500    8.240000    0.000000    0.000000    1.000000
50%      1.517680   13.300000    3.480000    1.360000   72.790000    0.555000    8.600000    0.000000    0.000000    2.000000
75%      1.519157   13.825000    3.600000    1.630000   73.087500    0.610000    9.172500    0.000000    0.100000    3

sumowanie się do 100%

In [149]:
print(glass.drop(['RI', 'Type'], axis=1).sum(axis=1))

0      99.82
1      99.89
2      99.78
3      99.59
4      99.83
       ...  
209    99.95
210    99.96
211    99.88
212    99.98
213    99.96
Length: 214, dtype: float64


Typy występujące w danych - brakuje typu: 4

In [150]:
print(glass['Type'].unique())

[1 2 3 5 6 7]


Średnie zawartości pierwiastów w danych typach
- typ 6: brak: Potasu, Baru i żelaza

In [151]:
print(glass.groupby('Type').mean())

            RI         Na        Mg        Al         Si         K         Ca        Ba        Fe
Type                                                                                             
1     1.518718  13.242286  3.552429  1.163857  72.619143  0.447429   8.797286  0.012714  0.057000
2     1.518619  13.111711  3.002105  1.408158  72.598026  0.521053   9.073684  0.050263  0.079737
3     1.517964  13.437059  3.543529  1.201176  72.404706  0.406471   8.782941  0.008824  0.057059
5     1.518928  12.827692  0.773846  2.033846  72.366154  1.470000  10.123846  0.187692  0.060769
6     1.517456  14.646667  1.305556  1.366667  73.206667  0.000000   9.356667  0.000000  0.000000
7     1.517116  14.442069  0.538276  2.122759  72.965862  0.325172   8.491379  1.040000  0.013448


Korelacje pomiędzy kolumnami

In [152]:
print(glass.corr())

            RI        Na        Mg        Al        Si         K        Ca        Ba        Fe      Type
RI    1.000000 -0.191885 -0.122274 -0.407326 -0.542052 -0.289833  0.810403 -0.000386  0.143010 -0.164237
Na   -0.191885  1.000000 -0.273732  0.156794 -0.069809 -0.266087 -0.275442  0.326603 -0.241346  0.502898
Mg   -0.122274 -0.273732  1.000000 -0.481799 -0.165927  0.005396 -0.443750 -0.492262  0.083060 -0.744993
Al   -0.407326  0.156794 -0.481799  1.000000 -0.005524  0.325958 -0.259592  0.479404 -0.074402  0.598829
Si   -0.542052 -0.069809 -0.165927 -0.005524  1.000000 -0.193331 -0.208732 -0.102151 -0.094201  0.151565
K    -0.289833 -0.266087  0.005396  0.325958 -0.193331  1.000000 -0.317836 -0.042618 -0.007719 -0.010054
Ca    0.810403 -0.275442 -0.443750 -0.259592 -0.208732 -0.317836  1.000000 -0.112841  0.124968  0.000952
Ba   -0.000386  0.326603 -0.492262  0.479404 -0.102151 -0.042618 -0.112841  1.000000 -0.058692  0.575161
Fe    0.143010 -0.241346  0.083060 -0.074402 -0.094201 

Po analizie
- klasyfikator bayesa zakładający niezależność cech może się nie sprawdzić ze względu na to, że jeśli czegoś będzie więcej to czegoś innego musi być mniej - w sumie nie może być więcej niż 100% przecież...

### 2. Przygotowanie danych
- podziel dane na zestaw uczący i walidacyjny (alternatywnie użyj walidacji krzyżowej),
- zbadaj wpływ różnego typu przetworzenia danych na wyniki klasyfikacji (proponowane: normalizacja, standaryzacja, dyskretyzacja, selekcja cech, PCA) - czyli wykonaj porównanie wyników bez przetworzenia danych z rezultatami po ich przetworzeniu, wykorzystując
co najmniej 2 metody różnego typu (osobno). (30 punktów)
- Bonus – usuń 5% wartości cech i przygotuj dane stosując metody radzenia sobie z brakującymi
danymi. (5 punktów)

usuń 5% cech i uzupełnij wartościami średnimi

In [153]:
glass_copy = glass.copy()
for col in glass.columns:
    if col == 'Type':
        continue
    glass_copy[col] = glass_copy[col].sample(frac=0.95, random_state=2137).to_frame().fillna(np.nan)

glass_mean_as_nan = pd.DataFrame(SimpleImputer(strategy='mean').fit_transform(glass_copy), columns=glass_copy.columns)
glass_mean_as_nan['Type'] = glass_mean_as_nan['Type'].astype(int)

print(glass_copy)
print(glass_mean_as_nan)
print(glass_copy.describe())
print(glass_mean_as_nan.describe())

          RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  Type
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0     1
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0     1
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0     1
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0     1
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0     1
..       ...    ...   ...   ...    ...   ...   ...   ...  ...   ...
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0     7
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0     7
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0     7
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0     7
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0     7

[214 rows x 10 columns]
          RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  Type
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0     1
1    1.51761  13.89  3.

podziel dane na zestaw uczący i walidacyjny (alternatywnie użyj walidacji krzyżowej)

In [154]:
X = glass.drop('Type', axis=1)
y = glass['Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2137)

zbadaj wpływ różnego typu przetworzenia danych na wyniki klasyfikacji (proponowane: normalizacja, standaryzacja, dyskretyzacja, selekcja cech, PCA) - czyli wykonaj porównanie wyników bez przetworzenia danych z rezultatami po ich przetworzeniu, wykorzystując co najmniej 2 metody różnego typu (osobno).

### Standaryzacja
- Standaryzacja przybliżyła średnią wartość każdej cechy do 0 oraz odchylenie standardowe do 1 - wartości cech są bardziej porównywalne.

In [155]:
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)
standarized_X_df = pd.DataFrame(X_train_standardized, columns=X_train.columns)

### Dyskretyzacja
- Dyskretyzacja podzieliła wartości cech na 100 przedziałów  - uzykujemy lepszą generalizację modelu, ponieważ nie będzie on zbytnio dopasowany do danych treningowych

In [156]:
# 100 binów, ordinal - zakodowanie jako liczby całkowite, uniform- równomierny podział przedziałów
discretizer = KBinsDiscretizer(n_bins=100, encode='ordinal', strategy='uniform')
X_train_discretized = discretizer.fit_transform(X_train)
X_test_discretized = discretizer.transform(X_test)
discretized_X_df = pd.DataFrame(X_train_discretized, columns=X_train.columns)

### Selekcja cech
- Selekcja cech pozwala na wybrania n najbardziej istotnych cech, dzięki czemu model będzie lepiej radził sobie z nowymi danymi, a także będzie szybciej się uczył. 2 najbardziej istotne cechy to magnez i bar

In [157]:
X = glass.drop('Type', axis=1)
y = glass['Type']
X = SelectKBest(chi2, k=2).fit_transform(X, y)
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X, y, test_size=0.1,
                                                                                        random_state=2137)
# dopisuję Mg i Ba po porównaniu z poprzednimi wynikami - Selekcja cech zwraca kolumny bez oznaczeń
selected_X_df = pd.DataFrame(X_train_selected, columns=['Mg', 'Ba'])


In [158]:
print(pd.DataFrame(X).describe())
print(X_train.describe())
print(standarized_X_df.describe())
print(discretized_X_df.describe())
print(selected_X_df.describe())

                0           1
count  214.000000  214.000000
mean     2.684533    0.175047
std      1.442408    0.497219
min      0.000000    0.000000
25%      2.115000    0.000000
50%      3.480000    0.000000
75%      3.600000    0.000000
max      4.490000    3.150000
               RI          Na          Mg          Al          Si           K          Ca          Ba          Fe
count  192.000000  192.000000  192.000000  192.000000  192.000000  192.000000  192.000000  192.000000  192.000000
mean     1.518350   13.389896    2.706667    1.448229   72.651354    0.515000    8.924375    0.179271    0.059583
std      0.002974    0.829210    1.441589    0.502838    0.780503    0.681026    1.392995    0.509434    0.100031
min      1.511150   10.730000    0.000000    0.290000   69.810000    0.000000    5.430000    0.000000    0.000000
25%      1.516528   12.887500    2.165000    1.190000   72.275000    0.137500    8.227500    0.000000    0.000000
50%      1.517685   13.260000    3.480000    1

### 3. Klasyfikacja
przetestuj klasyfikatory i zbadaj wpływ na wyniki:
- naiwny klasyfikator Bayesa
- drzewo decyzjne
używając przynajmniej 3 różnych zestawów hiperparametrów. (40 punktów)

Bonus – Przetestuj (ze zrozumieniem!) bardziej zaawansowane algorytmy, takie jak Las losowy
czy Klasyfikator wektorów nośnych (SVM, z ang. Support Vector Machines). (5 punktów)

### Naiwny bayes

In [159]:
smoothings = [0.005, 1e-3, 1e-15]

def classificationNB(X_train, X_test):
    models = []
    for smoothing in smoothings:
        nb = GaussianNB(var_smoothing=smoothing)
        nb.fit(X_train, y_train)
        y_pred = nb.predict(X_test)
        models.append((nb, y_pred))
    return models

fitted_models = classificationNB(X_train, X_test)
fitted_models_standarized = classificationNB(X_train_standardized, X_test_standardized)
fitted_models_discretized = classificationNB(X_train_discretized, X_test_discretized)

### Drzewo decyzyjne

In [160]:
criterion_params = ['gini', 'entropy']
max_depth_params = [None, 5, 10]

def classificationDT(X_train_p, X_test_p):
    models = []
    for criterion in criterion_params:
        for max_depth in max_depth_params:
            tree = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
            tree.fit(X_train_p, y_train)
            y_pred = tree.predict(X_test_p)
            models.append((tree, y_pred))
    return models

fitted_models_tree = classificationDT(X_train, X_test)
fitted_models_tree_standarized = classificationDT(X_train_standardized, X_test_standardized)
fitted_models_tree_discretized = classificationDT(X_train_discretized, X_test_discretized)


### 4. Ocena modeli

In [161]:
def evaluationNB(title, models):
    print(title)
    for i, (model, y_pred) in enumerate(models):
        print(f'Native Bayes - smoothing={smoothings[i]}')
        print(confusion_matrix(y_test, y_pred))
        print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
        print(f'Sensitivity: {recall_score(y_test, y_pred, average="macro")}')
        print(f'Precision: {precision_score(y_test, y_pred, average="macro")}')
        print(f'F1 score: {f1_score(y_test, y_pred, average="macro")}')
        print('\n')

evaluationNB('Naive Bayes with original data', fitted_models)
evaluationNB('Naive Bayes with standarized data', fitted_models_standarized)
evaluationNB('Naive Bayes with discretized data', fitted_models_discretized)

Naive Bayes with original data
Native Bayes - smoothing=0.005
[[6 0 0 0 0 0]
 [4 1 0 1 2 0]
 [3 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 2 0]
 [0 0 0 0 0 3]]
Accuracy: 0.5454545454545454
Sensitivity: 0.5208333333333334
Precision: 0.4935897435897436
F1 score: 0.4200779727095516


Native Bayes - smoothing=0.001
[[6 0 0 0 0 0]
 [4 1 0 1 2 0]
 [3 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 2 0]
 [0 0 0 0 0 3]]
Accuracy: 0.5454545454545454
Sensitivity: 0.5208333333333334
Precision: 0.4935897435897436
F1 score: 0.4200779727095516


Native Bayes - smoothing=1e-15
[[6 0 0 0 0 0]
 [4 2 0 2 0 0]
 [3 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 2 0]
 [0 0 0 0 0 3]]
Accuracy: 0.5909090909090909
Sensitivity: 0.5416666666666666
Precision: 0.576923076923077
F1 score: 0.5052631578947369


Naive Bayes with standarized data
Native Bayes - smoothing=0.005
[[6 0 0 0 0 0]
 [4 1 0 1 2 0]
 [3 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 2 0]
 [0 0 0 0 0 3]]
Accuracy: 0.5454545454545454
Sensitivity: 0.5208333333333334
Precision: 0.49358974

/Users/jakubradzik-vazco/Desktop/STUDIA SEM 6/Artificial_Intelligence_Lab/Lista4/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jakubradzik-vazco/Desktop/STUDIA SEM 6/Artificial_Intelligence_Lab/Lista4/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jakubradzik-vazco/Desktop/STUDIA SEM 6/Artificial_Intelligence_Lab/Lista4/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `

In [162]:
def evaluationDT(title, models):
    print(title)
    for i, (model, y_pred) in enumerate(models):
        print(f'Decision Tree - criterion={criterion_params[i // 3]} + max_depth={max_depth_params[i % 3]}')
        print(confusion_matrix(y_test, y_pred))
        print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
        print(f'Sensitivity: {recall_score(y_test, y_pred, average="macro")}')
        print(f'Precision: {precision_score(y_test, y_pred, average="macro")}')
        print(f'F1 score: {f1_score(y_test, y_pred, average="macro")}')
        print('\n')

evaluationDT('Decision Tree with original data', fitted_models_tree)
evaluationDT('Decision Tree with standardized data', fitted_models_tree_standarized)
evaluationDT('Decision Tree with discretized data', fitted_models_tree_discretized)

Decision Tree with original data
Decision Tree - criterion=gini + max_depth=None
[[5 0 1 0 0]
 [1 5 2 0 0]
 [1 0 2 0 0]
 [0 0 0 2 0]
 [0 0 0 0 3]]
Accuracy: 0.7727272727272727
Sensitivity: 0.825
Precision: 0.8228571428571427
F1 score: 0.8076923076923077


Decision Tree - criterion=gini + max_depth=5
[[6 0 0 0 0]
 [1 6 1 0 0]
 [1 0 2 0 0]
 [0 0 0 2 0]
 [0 0 0 0 3]]
Accuracy: 0.8636363636363636
Sensitivity: 0.8833333333333332
Precision: 0.8833333333333332
F1 score: 0.8761904761904763


Decision Tree - criterion=gini + max_depth=10
[[5 0 1 0 0]
 [0 6 2 0 0]
 [1 1 1 0 0]
 [0 0 0 2 0]
 [0 0 0 0 3]]
Accuracy: 0.7727272727272727
Sensitivity: 0.7833333333333334
Precision: 0.7880952380952382
F1 score: 0.7838095238095238


Decision Tree - criterion=entropy + max_depth=None
[[4 1 1 0 0]
 [3 4 1 0 0]
 [1 1 1 0 0]
 [0 0 0 2 0]
 [0 0 0 0 3]]
Accuracy: 0.6363636363636364
Sensitivity: 0.7
Precision: 0.7
F1 score: 0.6952380952380952


Decision Tree - criterion=entropy + max_depth=5
[[5 1 0 0 0]
 [3 5 0

In [163]:
data = []

def appendNB(models, name):
    for i, (_, y_pred) in enumerate(models):
        data.append(['Naive Bayes', name, smoothings[i], accuracy_score(y_test, y_pred),
                     recall_score(y_test, y_pred, average="macro"), precision_score(y_test, y_pred, average="macro"),
                     f1_score(y_test, y_pred, average="macro")])

def appendDT(models_p, name):
    for i, (_, y_pred) in enumerate(models_p):
        data.append(['Decision Tree', name, f'{criterion_params[i // 3]} - {max_depth_params[i % 3]}',
                     accuracy_score(y_test, y_pred), recall_score(y_test, y_pred, average="macro"),
                     precision_score(y_test, y_pred, average="macro"), f1_score(y_test, y_pred, average="macro")])
#
appendNB(fitted_models, 'Original')
appendNB(fitted_models_standarized, 'Standarized')
appendNB(fitted_models_discretized, 'Discretized')

appendDT(fitted_models_tree, 'Original')
appendDT(fitted_models_tree_standarized, 'Standarized')
appendDT(fitted_models_tree_discretized, 'Discretized')

df = pd.DataFrame(data, columns=['Model', 'Data', 'Parameters', 'Accuracy', 'Sensitivity', 'Precision', 'F1 score'])

df.sort_values(by=['Accuracy', 'Sensitivity', 'Precision', 'F1 score'], ascending=False, inplace=True)
print(df)

            Model         Data      Parameters  Accuracy  Sensitivity  Precision  F1 score
10  Decision Tree     Original        gini - 5  0.863636     0.883333   0.883333  0.876190
16  Decision Tree  Standarized        gini - 5  0.863636     0.883333   0.883333  0.876190
22  Decision Tree  Discretized        gini - 5  0.863636     0.883333   0.883333  0.876190
17  Decision Tree  Standarized       gini - 10  0.818182     0.850000   0.846667  0.838095
14  Decision Tree     Original    entropy - 10  0.818182     0.816667   0.821429  0.814615
18  Decision Tree  Standarized  entropy - None  0.818182     0.816667   0.821429  0.814615
9   Decision Tree     Original     gini - None  0.772727     0.825000   0.822857  0.807692
15  Decision Tree  Standarized     gini - None  0.772727     0.825000   0.822857  0.807692
20  Decision Tree  Standarized    entropy - 10  0.772727     0.791667   0.876190  0.793333
24  Decision Tree  Discretized  entropy - None  0.772727     0.791667   0.876190  0.793333

/Users/jakubradzik-vazco/Desktop/STUDIA SEM 6/Artificial_Intelligence_Lab/Lista4/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jakubradzik-vazco/Desktop/STUDIA SEM 6/Artificial_Intelligence_Lab/Lista4/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jakubradzik-vazco/Desktop/STUDIA SEM 6/Artificial_Intelligence_Lab/Lista4/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `